<img src='https://i.ibb.co/r7cthKF/dst-eda-5-4.png' width=600px/>

<hr>

# Cumulative tests

In [548]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [549]:
ab_data = pd.read_csv('data/ab_data.csv')
display(ab_data.head())
display(ab_data.info())

,user_id,timestamp,group,converted
0,851104,2017-01-21,A,0
1,804228,2017-01-12,A,0
2,661590,2017-01-11,B,0
3,853541,2017-01-08,B,0
4,864975,2017-01-21,A,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290482 entries, 0 to 290481
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    290482 non-null  int64 
 1   timestamp  290482 non-null  object
 2   group      290482 non-null  object
 3   converted  290482 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 8.9+ MB


None

In [550]:
ab_data['timestamp'] = pd.to_datetime(ab_data['timestamp'], format='%Y-%m-%d')

In [551]:
sample_ab = ab_data.groupby(['timestamp', 'group'], as_index=False).agg({
    'user_id': 'count',
    'converted': 'sum'
}).rename(columns={'user_id': 'users_count'})

sample_ab.head()

,timestamp,group,users_count,converted
0,2017-01-02,A,2851,357
1,2017-01-02,B,2848,339
2,2017-01-03,A,6584,749
3,2017-01-03,B,6620,754
4,2017-01-04,A,6573,797


In [552]:
sample_ab['conversion'] = sample_ab['converted'] / sample_ab['users_count'] * 100

sample_ab.head()
# data[data['timestamp'] == pd.to_datetime('2017-01-05')]

,timestamp,group,users_count,converted,conversion
0,2017-01-02,A,2851,357,12.521922
1,2017-01-02,B,2848,339,11.903090
2,2017-01-03,A,6584,749,11.376063
3,2017-01-03,B,6620,754,11.389728
4,2017-01-04,A,6573,797,12.125361


In [553]:
fig = px.box(
    sample_ab,
    y='group',
    x='conversion',
    color='group',
    points='all',
    title='Distribution of Conversion by groups'
)
fig.show()

In [554]:
median_mean = sample_ab.groupby('group')['conversion'].agg(['mean', 'median'])
display(median_mean)

,mean,median
group,,
A,12.032876,12.024261
B,11.872697,11.829582


In [555]:
fig = px.line(
    sample_ab,
    x='timestamp',
    y='conversion',
    color='group',
    title='Conversion plot by day'
)
fig.show()

In [556]:
sample_ab['cum_users_count'] = sample_ab.groupby(['group'])['users_count'].cumsum()
sample_ab['cum_converted']   = sample_ab.groupby(['group'])['converted'].cumsum()
sample_ab['cum_conversion']  = sample_ab['cum_converted'] / sample_ab['cum_users_count'] * 100

sample_ab.head()

,timestamp,group,users_count,converted,conversion,cum_users_count,cum_converted,cum_conversion
0,2017-01-02,A,2851,357,12.521922,2851,357,12.521922
1,2017-01-02,B,2848,339,11.903090,2848,339,11.903090
2,2017-01-03,A,6584,749,11.376063,9435,1106,11.722311
3,2017-01-03,B,6620,754,11.389728,9468,1093,11.544149
4,2017-01-04,A,6573,797,12.125361,16008,1903,11.887806


In [557]:
fig = px.line(
    sample_ab,
    x='timestamp',
    y='cum_conversion',
    color='group',
    title='Cumulative conversion plot by day'
)
fig.show()

<hr>

In [558]:
sample_a = pd.read_csv('Data/ab_test-redesign_sample_a.csv')
sample_b = pd.read_csv('Data/ab_test-redesign_sample_b.csv')

display(sample_a.info())
display(sample_a.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61571 entries, 0 to 61570
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          61571 non-null  object 
 1   cid           61571 non-null  float64
 2   transactions  61571 non-null  int64  
 3   revenue       61571 non-null  int64  
 4   group         61571 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 2.3+ MB


None

,date,cid,transactions,revenue,group
0,2020-11-06,1.001648e+09,0,0,A
1,2020-11-06,1.001936e+09,0,0,A
2,2020-11-06,1.003214e+08,0,0,A
3,2020-11-06,1.004393e+09,0,0,A
4,2020-11-06,1.006382e+09,0,0,A


In [559]:
sample_a['date'] = pd.to_datetime(sample_a['date'], format='%Y-%m-%d')
sample_b['date'] = pd.to_datetime(sample_b['date'], format='%Y-%m-%d')

sample = pd.concat([sample_a, sample_b], axis=0).sort_values(by=['date', 'group'])
sample.head()

,date,cid,transactions,revenue,group
0,2020-11-06,1.001648e+09,0,0,A
1,2020-11-06,1.001936e+09,0,0,A
2,2020-11-06,1.003214e+08,0,0,A
3,2020-11-06,1.004393e+09,0,0,A
4,2020-11-06,1.006382e+09,0,0,A


In [560]:
sample_grouped = sample.groupby(['date', 'group'], as_index=False).agg({
    'cid': 'count',
    'transactions': 'sum',
    'revenue': 'sum'
}).rename(columns={
    'cid': 'users_count',
    'transactions': 'transactions',
    'revenue': 'revenues'
})
sample_grouped.head()

,date,group,users_count,transactions,revenues
0,2020-11-06,A,2576,16,1577279
1,2020-11-06,B,2499,19,996645
2,2020-11-07,A,2704,23,1721886
3,2020-11-07,B,2804,19,555499
4,2020-11-08,A,3354,25,846051


In [561]:
sample_grouped['mean_transaction'] = sample_grouped['revenues'] / sample_grouped['transactions']

sample_grouped['cum_users_count']      = sample_grouped.groupby(['group'])['users_count'].cumsum()
sample_grouped['cum_transactions']     = sample_grouped.groupby(['group'])['transactions'].cumsum()
sample_grouped['cum_revenues']         = sample_grouped.groupby(['group'])['revenues'].cumsum() 

sample_grouped['cum_conversion']       = sample_grouped['cum_transactions'] / sample_grouped['cum_users_count'] * 100
sample_grouped['cum_mean_transaction'] = sample_grouped['cum_revenues'] / sample_grouped['cum_transactions']

sample_grouped.head()

,date,group,users_count,transactions,revenues,mean_transaction,cum_users_count,cum_transactions,cum_revenues,cum_conversion,cum_mean_transaction
0,2020-11-06,A,2576,16,1577279,98579.937500,2576,16,1577279,0.621118,98579.937500
1,2020-11-06,B,2499,19,996645,52455.000000,2499,19,996645,0.760304,52455.000000
2,2020-11-07,A,2704,23,1721886,74864.608696,5280,39,3299165,0.738636,84593.974359
3,2020-11-07,B,2804,19,555499,29236.789474,5303,38,1552144,0.716576,40845.894737
4,2020-11-08,A,3354,25,846051,33842.040000,8634,64,4145216,0.741256,64769.000000


In [562]:
fig = px.line(
    sample_grouped,
    x='date',
    y='cum_conversion',
    color='group',
     title='Cumulative conversion plot by day'
)
fig.show()

In [563]:
fig = px.line(
    sample_grouped,
    x='date',
    y='cum_mean_transaction',
    color='group',
     title='Cumulative average check plot by day'
)
fig.show()

<hr>

# Stat tests

In [564]:
from statsmodels.stats.proportion import proportions_ztest

In [565]:
converted_piv= ab_data.groupby('group')['converted'].agg(
    ['sum', 'count']
)

H0 = 'Conversions in groups A and B are equal'
H1 = 'The conversion rate in Group A is higher than the conversion rate in Group B'

alpha  = 0.05

_, p = proportions_ztest(
    count=converted_piv['sum'],
    nobs=converted_piv['count'],
    alternative='larger'
)

print(f"p={'%.3f' % p}: {H0 if p > alpha else H1}")

p=0.095: Conversions in groups A and B are equal


In [566]:
converted_piv= sample.groupby('group').agg({
    'transactions': 'sum', 
    'cid': 'count'
}).rename(columns={
    'transactions': 'sum',
    'cid': 'count'
})

H0 = 'Conversions in groups A and B are equal'
H1 = 'The conversion rate in Group A is less than the conversion rate in Group B'

alpha  = 0.1

_, p = proportions_ztest(
    count=converted_piv['sum'],
    nobs=converted_piv['count'],
    alternative='smaller'
)

print(f"p={'%.3f' % p}: {H0 if p > alpha else H1}")

p=0.218: Conversions in groups A and B are equal


In [567]:
from scipy.stats import shapiro

In [568]:
H0 = 'Mean_transaction is distributed normally'
Ha = 'Mean_transaction is not distributed normally'

alpha = 0.1

_, pa = shapiro(sample_grouped[sample_grouped['group'] == 'A']['mean_transaction'])
_, pb = shapiro(sample_grouped[sample_grouped['group'] == 'B']['mean_transaction'])

print(f"pa={'%.3f' % pa}: {H0 if pa > alpha else Ha} in group A")
print(f"pb={'%.3f' % pb}: {H0 if pb > alpha else Ha} in group B")

pa=0.010: Mean_transaction is not distributed normally in group A
pb=0.022: Mean_transaction is not distributed normally in group B


In [569]:
from scipy.stats import mannwhitneyu

In [570]:
H0 = 'The daily average check in Group A is equal to check in Group B'
Ha = 'The daily average check in Group A is less than in Group B'

alpha = 0.1

_, p = mannwhitneyu(
    sample_grouped[sample_grouped['group'] == 'A']['mean_transaction'],
    sample_grouped[sample_grouped['group'] == 'B']['mean_transaction'],
    alternative='less'
)

print(f"p={'%.3f' % p}: {H0 if p > alpha else Ha}")

p=0.089: The conversion rate in Group A is less than the conversion rate in Group B


<hr>

# Confidence interval

## For mean

In [571]:
from scipy.stats import norm
from scipy.stats import t

def conf_interval_std(n, x_mean, sigma, gamma=0.95):
    alpha = 1 - gamma # significance level
    z_crit = round(-norm.ppf(alpha/2), 3) # z critical
    
    print(f"z_crit: {'%.2f' % z_crit}")
    
    eps = z_crit * sigma/(n ** 0.5) # eps
    lower_bound = x_mean - eps # lower bound
    upper_bound = x_mean + eps # upper bound
    
    return (round(lower_bound, 2), round(upper_bound, 2))

def conf_interval(n, x_mean, x_std, gamma=0.95):
    alpha = 1 - gamma # significance level
    k = n - 1 # degree of freedom
    t_crit = -t.ppf(alpha/2, k) # t-критическое
    
    print(f"t_crit: {'%.2f' % t_crit}")
    
    eps = t_crit * x_std/(n ** 0.5) # eps
    lower_bound = x_mean - eps # lower bound
    upper_bound = x_mean + eps # upper bound
    
    return (round(lower_bound, 2), round(upper_bound, 2))
    
n = 64 # sample size
x_mean = 12.35 # sample mean
sigma = 2.4 # true std
gamma = 0.9 # reliability level

print('Confidence interval: {}'.format(conf_interval_std(n, x_mean, sigma, gamma)))

n = 25 # sample size
x_mean = 3540 # sample mean
x_std = 1150 # std
gamma = 0.9 # reliability level

print('Confidence interval: {}'.format(conf_interval(n, x_mean, x_std, gamma)))

z_crit: 1.65
Confidence interval: (11.86, 12.84)
t_crit: 1.71
Confidence interval: (3146.5, 3933.5)


## For proportions

In [572]:
a_data = ab_data[ab_data['group'] == 'A']
b_data = ab_data[ab_data['group'] == 'B']

def proportions_conf_interval(n, x_p, gamma=0.95):   
    alpha = 1 - gamma # significance level
    z_crit = -norm.ppf(alpha/2) # z critical
    
    print(f"z_crit: {'%.2f' % z_crit}")
    
    eps = z_crit * (x_p * (1 - x_p) / n) ** 0.5 # eps
    lower_bound = x_p - eps # lower bound
    upper_bound = x_p + eps # upper bound
    
    return round(lower_bound * 100, 2), round(upper_bound * 100, 2)

conf_interval_a = proportions_conf_interval(
    n=a_data['user_id'].count(), 
    x_p=a_data['converted'].mean() 
)
conf_interval_b = proportions_conf_interval(
    n=b_data['user_id'].count(), 
    x_p=b_data['converted'].mean()
)

print('Confidence interval for group A conversion: {}'.format(conf_interval_a))
print('Confidence interval for group B conversion: {}'.format(conf_interval_b))

z_crit: 1.96
z_crit: 1.96
Confidence interval for group A conversion: (11.86, 12.19)
Confidence interval for group B conversion: (11.7, 12.03)


In [573]:
def diff_proportions_conf_interval(n, xp, gamma=0.95):
    alpha = 1 - gamma  # significance level
    diff = xp[1] - xp[0] # sample conversion diff
    z_crit = -norm.ppf(alpha/2) # z critical
    
    print(f"z_crit: {'%.2f' % z_crit}")
    
    eps = z_crit * (xp[0] * (1 - xp[0])/n[0] + xp[1] * (1 - xp[1])/n[1]) ** 0.5 # eps
    lower_bound = diff - eps # lower bound
    upper_bound = diff + eps # upper bound
    
    return round(lower_bound *100, 2), round(upper_bound * 100, 2)

n = [a_data['user_id'].count(), b_data['user_id'].count()]
x_p = [a_data['converted'].mean(), b_data['converted'].mean()]

print('Confidence interval for conversion diff: {}'.format(diff_proportions_conf_interval(n, x_p)))

z_crit: 1.96
Confidence interval for conversion diff: (-0.39, 0.08)


<hr>

In [581]:
converted_piv = updated_data.groupby('group')['purchase'].agg(
    ['sum', 'count']
)

H0 = 'Conversions in groups A and B are equal'
H1 = 'The conversion rate in Group A is higher than the conversion rate in Group B'

alpha  = 0.05

_, p = proportions_ztest(
    count=converted_piv['sum'],
    nobs=converted_piv['count'],
    alternative='larger'
)

print(f"p={'%.3f' % p}: {H0 if p > alpha else H1}")

p=0.120: Conversions in groups A and B are equal


In [582]:
a = updated_data[updated_data['group'] == 'A']
b = updated_data[updated_data['group'] == 'B']

grouped_a = grouped[grouped['group'] == 'A']
grouped_b = grouped[grouped['group'] == 'B']

In [583]:
conf_interval_a = proportions_conf_interval(
    n=a['user_id'].count(), 
    x_p=a['purchase'].mean() 
)
conf_interval_b = proportions_conf_interval(
    n=b['user_id'].count(), 
    x_p=b['purchase'].mean()
)

print('Confidence interval for group A conversion: {}'.format(conf_interval_a))
print('Confidence interval for group B conversion: {}'.format(conf_interval_b))

n = [a['user_id'].count(), b['user_id'].count()]
x_p = [a['purchase'].mean(), b['purchase'].mean()]

print('Confidence interval for conversion diff: {}'.format(diff_proportions_conf_interval(n, x_p)))

z_crit: 1.96
z_crit: 1.96
Confidence interval for group A conversion: (11.87, 12.2)
Confidence interval for group B conversion: (11.73, 12.06)
z_crit: 1.96
Confidence interval for conversion diff: (-0.38, 0.09)


In [584]:
H0 = 'mean_check is distributed normally'
Ha = 'mean_check is not distributed normally'

alpha = 0.05

_, pa = shapiro(grouped[grouped['group'] == 'A']['mean_check'])
_, pb = shapiro(grouped[grouped['group'] == 'B']['mean_check'])

print(f"pa={'%.3f' % pa}: {H0 if pa > alpha else Ha} in Group A")
print(f"pb={'%.3f' % pb}: {H0 if pb > alpha else Ha} in Group B")

pa=0.253: mean_check is distributed normally in Group A
pb=0.094: mean_check is distributed normally in Group B


In [585]:
from scipy.stats import ttest_ind

In [586]:
H0 = 'The daily mean check in Group A is equal to check in Group B'
Ha = 'The daily mean check in Group A is less than in Group B'

alpha = 0.05

_, p = ttest_ind(
    grouped[grouped['group'] == 'A']['mean_check'],
    grouped[grouped['group'] == 'B']['mean_check'],
    alternative='less'
)

print(f"p={'%.3f' % p}: {H0 if p > alpha else H1}")

p=0.000: The conversion rate in Group A is higher than the conversion rate in Group B


In [587]:
n_a = grouped_a.shape[0] # sample size
x_mean_a = grouped_a['mean_check'].mean() # sample mean
x_std_a = grouped_a['mean_check'].std() # std

n_b = grouped_b.shape[0] # sample size
x_mean_b = grouped_b['mean_check'].mean() # sample mean
x_std_b = grouped_b['mean_check'].std() # std

print('Confidence interval of mean check in Group A: {}'.format(conf_interval(n_a, x_mean_a, x_std_a)))
print('Confidence interval of mean check in Group A: {}'.format(conf_interval(n_b, x_mean_b, x_std_b)))

t_crit: 2.07
Confidence interval of mean check in Group A: (79630.35, 82308.22)
t_crit: 2.07
Confidence interval of mean check in Group A: (87625.83, 89525.61)
